<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NAML: Neural News Recommendation with Attentive Multi-View Learning
NAML \[1\] is a multi-view news recommendation approach. The core of NAML is a news encoder and a user encoder. The newsencoder is composed of a title encoder, a body encoder, a vert encoder and a subvert encoder. The CNN-based title encoder and body encoder learn title and body representations by capturing words semantic information. After getting news title, body, vert and subvert representations, an attention network is used to aggregate those vectors. In the user encoder, we learn representations of users from their browsed news. Besides, we apply additive attention to learn more informative news and user representations by selecting important words and news.

## Properties of NAML:
- NAML is a multi-view neural news recommendation approach.
- It uses news title, news body, news vert and news subvert to get news repersentations. And it uses user historical behaviors to learn user representations.
- NAML uses additive attention to learn informative news and user representations by selecting important words and news.

## Data format:

### train data
One simple example: <br>

`1 0 0 0 0 Impression:0 User:502 CandidateTitle0:17917,36557,47926,32224,24113,48923,19086,5636,3703,0... CandidateBody0:17024,53305,8832,29800,9787,4068,48731,48923,19086,38699,5766,22487,38336,29800,8548,39128,33457,7789,
30543,7482,8548,49004,53305,22999,32747,21103,11799,5766,4868,17115,7482,15118,48731,2025,7789,23336,7789,48731,19086,
10630,11128,36557,3703,47354,611,7789,19086,5636,51521,30706... CandidateVert0:14... CandidateSubvert0:219... ClickedTitle0:48,33405,35198,5969,5636,35845,850,48731,46799,24113... ClickedBody0:36557,67,34519,24113,8548,48,33405,35198,5969,14340,7053,850,8823,9498,46799,24113,12506,32747,31130,
3074,48731,20869,14264,38289,37310,7789,36557,34967,48731,36916,23321,3595,48731,47354,4868,15719,7482,12771,50693,
47354,17523,48,20918,17900,35198,48731,20869,1220,14264,7789... ClickedVert0:14... ClickedSubvert0:99... `
<br>

In general, each line in data file represents one positive instance and n negative instances in a same impression. The format is like: <br>

`[label0] ... [labeln] [Impression:i] [User:u] [CandidateTitle0:w1,w2,w3,...] ... [CandidateBody0:w1,w2 ..] ... [CandidateVert0:v] ... [CandidateSubvert0:s] ... [ClickedTitle0:w1,w2,w3,...] ... [ClickedBody0:w1,w2,w3,...] ... [ClickedVert0:v] ... [ClickedSubvert0:s] ...`

<br>

It contains several parts seperated by space, i.e. label part, Impression part `<impresison id>`, User part `<user id>`, CandidateNews part, ClickedHistory part. CandidateNews part describes the target news article we are going to score in this instance. It is represented by (aligned) title words, body words, news vertical index and subvertical index. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateTitlei:34,45,334,23,12,987,3456,111,456,432`. <br>
ClickedHistory describe the k-th news article the user ever clicked and the format is the same as candidate news. Every clicked news has title words, body words, vertical and subvertical. We use a fixed length to describe an article, if the title or body is less than the fixed length, just pad it with zeros.

### test data
One simple example: <br>
`1 Impression:0 User:1529 CandidateTitle0:5327,18658,13846,6439,611,50611,0,0,0,0 CandidateBody0:13846,3197,27902,225,5327,45008,29145,7789,509,7395,11502,36557,13846,23680,26492,38072,20507,5636,
4247,32747,50132,7482,41049,32747,43022,50611,35979,7789,1191,36557,52870,21622,48148,42737,48731,36557,13846,23680,
13173,7482,13848,38072,20507,7789,41675,36875,51461,12348,21045,42160 CandidateVert0:14 CandidateSubvert0:19 ClickedTitle0:9079,3703,32747,8546,19377,50184,32747,24026,40010,49754 ... ClickedBody0:26061,48731,8576,7789,8683,9079,5636,45084,46805,3703,509,43036,11502,28883,9498,18450,32747,8546,33405,
35647,50184,7482,41143,8220,43618,38072,35198,43390,28057,32552,45245,10764,16247,4221,41038,36557,43683,46805,7789,
29727,2179,51003,34797,897,21045,12974,23382,46287,48731,15206 ... ClickedVert0:14 ... ClickedSubvert0:219 ...`
<br>

In general, each line in data file represents one instance. The format is like: <br>

`[label] [Impression:i] [User:u] [CandidateTitle0:w1,w2,w3,...] [CandidateBody0:w1,w2,w3,...] [CandidateVert0:v] [CandidateSubvert0:s] [ClickedTitle0:w1,w2,w3,...] ... [ClickedBody0:w1,w2,w3,...] ... [ClickedVert0:v] ... [ClickedSubvert0:s] ...`
<br>

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.naml import NAMLModel
from reco_utils.recommender.newsrec.io.naml_iterator import NAMLIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf
import os

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) 
[GCC 7.3.0]
Tensorflow version: 1.15.2


## Download and load data

In [2]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'naml.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', data_path, 'naml.zip')

100%|██████████| 72.6k/72.6k [00:04<00:00, 16.7kKB/s]


## Create hyper-parameters

In [3]:
epochs=5
seed=42

In [4]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epochs)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

data_format=naml,iterator_type=None,wordEmb_file=/tmp/tmp37nn3h_g/embedding.npy,doc_size=None,title_size=10,body_size=50,word_emb_dim=100,word_size=54071,user_num=10329,vert_num=17,subvert_num=249,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=100,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=relu,filter_num=200,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=50,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=5,batch_size=64,show_step=100000,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


In [5]:
iterator = NAMLIterator

## Train the NAML model

In [6]:
model = NAMLModel(hparams, iterator, seed=seed)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
print(model.run_eval(valid_file))


{'group_auc': 0.533, 'mean_mrr': 0.1792, 'ndcg@5': 0.177, 'ndcg@10': 0.2417}


In [8]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:1.5924944522429485
eval info: group_auc:0.5565, mean_mrr:0.1811, ndcg@10:0.2487, ndcg@5:0.1817
at epoch 1 , train time: 46.0 eval time: 38.3
at epoch 2
train info: logloss loss:1.5695580341378037
eval info: group_auc:0.5568, mean_mrr:0.1817, ndcg@10:0.2475, ndcg@5:0.1769
at epoch 2 , train time: 42.9 eval time: 38.1
at epoch 3
train info: logloss loss:1.5507925962915226
eval info: group_auc:0.5633, mean_mrr:0.1884, ndcg@10:0.2528, ndcg@5:0.1943
at epoch 3 , train time: 42.9 eval time: 38.2
at epoch 4
train info: logloss loss:1.5379242575898462
eval info: group_auc:0.5661, mean_mrr:0.1933, ndcg@10:0.256, ndcg@5:0.1952
at epoch 4 , train time: 42.8 eval time: 38.2
at epoch 5
train info: logloss loss:1.5233123536012612
eval info: group_auc:0.5666, mean_mrr:0.1919, ndcg@10:0.2574, ndcg@5:0.1955
at epoch 5 , train time: 42.9 eval time: 38.3


In [9]:
res_syn = model.run_eval(valid_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'group_auc': 0.5666, 'mean_mrr': 0.1919, 'ndcg@5': 0.1955, 'ndcg@10': 0.2574}


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: Neural News Recommendation with Attentive Multi-View Learning, IJCAI 2019<br>